Question 1:

What's the version.build_hash value?

42f05b9372a9a4a470db3b52817899b99a76ee73

In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
from elasticsearch import Elasticsearch

In [6]:
es_client = Elasticsearch('http://localhost:9200')

In [7]:
es_client.info()

ObjectApiResponse({'name': '74e137d60c6e', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'OEQI1DUkRkm7gXDK4QJ4FQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

Question 2:

Which function do you use for adding your data to elastic?

Index

In [8]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "docker-question"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'docker-question'})

Question 3:

What's the score for the top ranking result?

75.54128

In [10]:
from tqdm.auto import tqdm

/opt/anaconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:02<00:00, 337.62it/s]


In [12]:
query = 'How do I execute a command in a running docker container?'

In [19]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [20]:
response = es_client.search(index=index_name, body=search_query)

In [21]:
response

ObjectApiResponse({'took': 34, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 390, 'relation': 'eq'}, 'max_score': 75.54128, 'hits': [{'_index': 'docker-question', '_id': '4wCTbpABpszt5eBLu2K2', '_score': 75.54128, '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)\nVersion: 4.0.1\nHome: http://pgcli.com\nroot@pg-database:ny_taxi> \\dt\n+--------+------------------+-------+-------+\n| Schema | Name             | Type  | Owner |\n|

In [22]:
search_ml_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [28]:
response_ml = es_client.search(index=index_name, body=search_ml_query)

In [31]:
result_docs = []

for hit in response_ml['hits']['hits']:
    result_docs.append(hit['_source']) 


In [32]:
result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [33]:
def elastic_search_ml(query):

    search_ml_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response_ml = es_client.search(index=index_name, body=search_ml_query)
    
    result_docs = []
    
    for hit in response_ml['hits']['hits']:
        result_docs.append(hit['_source']) 
    
    return result_docs

Question 4:

Now let's only limit the questions to machine-learning-zoomcamp.

Return 3 results. What's the 3rd question returned by the search engine?



In [34]:
elastic_search_ml(query)

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

Question 5:

What's the length of the resulting prompt? (use the len function)

In [45]:
from openai import OpenAI

In [47]:
%load_ext dotenv
%dotenv

In [48]:
client = OpenAI()

In [49]:
query = 'How do I execute a command in a running docker container?'

In [50]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer that QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION. 

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"question: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [51]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
)

    return response.choices[0].message.content

In [52]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [53]:
rag(query)

'To execute a command in a running Docker container, you can either attach to the container and then run the command from within it, or use the `docker exec` command to run the command from outside the container. Based on the available context:\n\nFor running `pgcli` inside a Docker container using the values provided in the context, you can use the following command:\n\n```sh\ndocker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n```\n\nThis command will run a new container with the specified network settings, and you can interact with `pgcli` inside it:\n```sh\npgcli -h pg-database -U root -p 5432 -d ny_taxi\n```\n\nHere, `pgcli` is used to connect to a PostgreSQL database within the Docker container.\n\nIf you need to execute any other command within a running container, you can use the `docker exec` command. For example, if the container ID is `175dd47cda07`, you can execute a command like this:\n\n```sh\ndocker exec -it 175dd47cda07 your_command_here\n```\n\nRepla

In [54]:
len(rag(query))

1287

Question 6:

Use the encode function. How many tokens does our prompt have?,

In [58]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 845.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 4.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.7/281.7 kB 5.4 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [60]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

In [65]:
search_results = elastic_search(query)
prompt = build_prompt(query, search_results)

In [69]:
len(encoding.encode(prompt))

735